# Cells
A notebook is a list of cells. Cells contain either explanatory text or executable code and its output. Click a cell to select it.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# New Section

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import keras
from keras import layers, models
from keras.models import load_model, Model
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
import cv2
#from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Input, Activation, merge, Convolution2D, UpSampling2D
from keras.layers.merge import concatenate, add
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import backend as K
import tensorflow as tf

width = 704
#width = 720
height = 1152
#height = 1160

channel = 1
batch_size = 5
num_epochs = 10


Using TensorFlow backend.


In [0]:
def dice_coef(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)
def bce_dice_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)

def keras_model(img_width=256, img_height=256):

    n_ch_exps = [4, 5, 6, 7, 8, 9]   #the n-th deep channel's exponent i.e. 2**n 16,32,64,128,256
    k_size = (3, 3)                  #size of filter kernel
    k_init = 'he_normal'             #kernel initializer
  
    ch_axis = 3
    input_shape = (img_width, img_height, 1)

    inp = Input(shape=input_shape)
    encodeds = []

    # encoder
    enc = inp
    print(n_ch_exps)
    for l_idx, n_ch in enumerate(n_ch_exps):
        enc = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(enc)
        enc = Dropout(0.1*l_idx,)(enc)
        enc = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(enc)
        encodeds.append(enc)
        #print(l_idx, enc)
        if n_ch < n_ch_exps[-1]:  #do not run max pooling on the last encoding/downsampling step
            enc = MaxPooling2D(pool_size=(2,2))(enc)
    
    # decoder
    dec = enc
    print(n_ch_exps[::-1][1:])
    decoder_n_chs = n_ch_exps[::-1][1:]
    for l_idx, n_ch in enumerate(decoder_n_chs):
        l_idx_rev = len(n_ch_exps) - l_idx - 2  #
        dec = Conv2DTranspose(filters=2**n_ch, kernel_size=k_size, strides=(2,2), activation='relu', padding='same', kernel_initializer=k_init)(dec)
        dec = concatenate([dec, encodeds[l_idx_rev]], axis=ch_axis)
        dec = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(dec)
        dec = Dropout(0.1*l_idx)(dec)
        dec = Conv2D(filters=2**n_ch, kernel_size=k_size, activation='relu', padding='same', kernel_initializer=k_init)(dec)

    outp = Conv2DTranspose(filters=1, kernel_size=k_size, activation='sigmoid', padding='same', kernel_initializer='glorot_normal')(dec)

    model = Model(inputs=[inp], outputs=[outp])
    
    return model

#============================================================
def get_data(cvs_data,batch_size=10):
    import random
    #print ("OK",cvs_data,batch_size)
    batches = int(cvs_data.shape[0] / batch_size)
    x_batch_filelist = []
    imgpath = '/content/drive/My Drive/NTHU/HW03/ccpd6000/train_images/'
    width = 704
    height = 1152
    while 1:
        for i in range(batches):
            #reset
            x_batch_filelist=[]
            np.random.shuffle(cvs_data)
            y_data = np.hstack((np.array(cvs_data[:, 1])[:, np.newaxis], np.array(cvs_data[:, 2])[:, np.newaxis]))
            for i in range(3, 9):
                y_data = np.hstack((y_data, np.array(cvs_data[:, i])[:, np.newaxis]))

            for i in range(0, batch_size, 1):
                x_batch_filelist.append(imgpath + cvs_data[i, 0])
            print("x_batch_filelist : ", x_batch_filelist)

            y_batch = y_data[:batch_size, :]
            y_batch = y_batch.astype(np.float64)
            y_batch = y_batch.astype(np.int_)

            # 1205 y_batch from csv cols (n,8)  to image array (n,H,W)
            y_batch_img = np.zeros((y_batch.shape[0], height, width, 1), dtype=np.uint8)
            for i in range(0, y_batch.shape[0], 1):
                y_batch_img_t = np.zeros((height, width, 1), np.uint8)
                y_batch_rs= np.reshape(y_batch[i, :], (4, 2))   #reshaape to cv.fill format
                #1208 
                #cv2.fillConvexPoly(y_batch_img_t, y_batch_rs, 255)
                #cv2.circle(y_batch_img_t, (int(y_batch_rs[0][0]), int(y_batch_rs[0][1])), 2, 255, 2, 1)
                #cv2.circle(y_batch_img_t, (int(y_batch_rs[1][0]), int(y_batch_rs[1][1])), 2, 255, 2, 1)
                #cv2.circle(y_batch_img_t, (int(y_batch_rs[2][0]), int(y_batch_rs[2][1])), 2, 255, 2, 1)
                #cv2.circle(y_batch_img_t, (int(y_batch_rs[3][0]), int(y_batch_rs[3][1])), 2, 255, 2, 1)
                y_batch_img[i,:,:,:] = y_batch_img_t[:,:,:]

            #x_batch = np.array([np.array(Image.open(fname)) for fname in x_batch_filelist])
            x_batch = np.array([np.array(Image.open(fname).convert('L')) for fname in x_batch_filelist])
            # 1205 w 720 * h 1160 = > w 704 * h 1152 (U-net trans. needed)
            x_batch = x_batch [:,0:height,0:width]
            # check label and img match
            #cv2.circle(x_batch[0], (int(y_batch[0, 0]*720), int(y_batch[0, 1]*1160)), 20, 255, 5)
            #cv2.imshow(str(x_batch_filelist[0]), x_batch[0])
            #cv2.imshow("AA", y_batch_img[0])
            #cv2.waitKey(0)
            #cv2.destroyAllWindows()

            x_batch = x_batch.reshape(x_batch.shape[0],x_batch.shape[1],x_batch.shape[2],channel)
            x_batch = x_batch / 255.0
            y_batch_img = y_batch_img.reshape(y_batch_img.shape[0], y_batch_img.shape[1], y_batch_img.shape[2], channel)
            y_batch_img = y_batch_img / 255.0

            print ("OK")
            yield(x_batch,y_batch_img)



In [7]:
csv_all = np.loadtxt("/content/drive/My Drive/NTHU/HW03/ccpd6000/train.csv", dtype=np.str, delimiter=',', skiprows=1,usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8))
csv_train, csv_valid = train_test_split(csv_all, test_size=0.1, random_state=42, shuffle=True)

model = keras_model(img_width=height, img_height=width)
optimizer = 'adam'
loss      = bce_dice_loss
metrics   = ['accuracy',mean_iou]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.summary()

filepath="/content/drive/My Drive/NTHU/HW03/weights-1206E10.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=False)

model.fit_generator(generator = get_data(csv_train,batch_size),steps_per_epoch = int(csv_train.shape[0]/batch_size),
                    epochs = num_epochs,verbose = 1,callbacks =[checkpoint],
                    validation_data = get_data(csv_valid,batch_size),
                    validation_steps = 2)

model.save('/content/drive/My Drive/NTHU/HW03/Unet_model_1206.h5')




[4, 5, 6, 7, 8, 9]



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[8, 7, 6, 5, 4]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.






Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1152, 704, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1152, 704, 16 160         input_1[0][0]                    
__________________________________________________________________________________________________
drop

KeyboardInterrupt: ignored